# Exploration

In this file, I'll be exploring the NEON API using [this NEON tutorial](https://www.neonscience.org/resources/learning-hub/tutorials/neon-api-01-introduction-requests)

In [80]:
import requests
import json

In [81]:
# To request data, need to provide NEON's URL
SERVER = 'http://data.neonscience.org/api/v0/'

Some of the important endpoints for NEON are:
* _sites/_
* _products/_
* _data/_

In [82]:
# Define sites of interest
SITECODE_LIST = ['TEAK', 'SOAP', 'SJER']

# Make requests, then turn request data into a Python JSON object so it's easier to interact with
site_json_list = [requests.get(SERVER+'sites/'+SITECODE).json() 
                          for SITECODE in SITECODE_LIST]

In [83]:
# Display json info TEAK site
TEAK_json = site_json_list[0]
TEAK_json

{'data': {'siteCode': 'TEAK',
  'siteName': 'Lower Teakettle NEON',
  'siteDescription': 'Lower Teakettle NEON',
  'siteType': 'GRADIENT',
  'siteLatitude': 37.00583,
  'siteLongitude': -119.00602,
  'stateCode': 'CA',
  'stateName': 'California',
  'domainCode': 'D17',
  'domainName': 'Pacific Southwest',
  'deimsId': 'https://deims.org/16d02952-7cb5-487b-a5a3-7bc9ba0d557e',
  'releases': [{'release': 'RELEASE-2021',
    'generationDate': '2021-01-23T02:30:02Z',
    'url': 'https://data.neonscience.org/api/v0/releases/RELEASE-2021'},
   {'release': 'RELEASE-2022',
    'generationDate': '2022-01-20T17:39:46Z',
    'url': 'https://data.neonscience.org/api/v0/releases/RELEASE-2022'}],
  'dataProducts': [{'dataProductCode': 'DP1.00001.001',
    'dataProductTitle': '2D wind speed and direction',
    'availableMonths': ['2018-06',
     '2018-07',
     '2018-08',
     '2018-09',
     '2018-10',
     '2018-11',
     '2018-12',
     '2019-01',
     '2019-02',
     '2019-03',
     '2019-04',
  

That was a huge dict...let's explore smaller parts and try to avoid listing the many dates

In [84]:
print(TEAK_json.keys(),'\n') # original dict only has 1 key -- 'data'

# look at info inside of 'data' key
NEON_dataDict_info = [str(key)+':\t'+str(type(TEAK_json['data'][key])) 
                          for key in TEAK_json['data'].keys()] 
print('\n'.join(NEON_dataDict_info), '\n')


# data products looked most interesting because I want to use that list to query products of interest
print('Exploring dataProducts')
dataProduct_Info = [str(type(product)) 
                        for product in TEAK_json['data']['dataProducts']] 
print('dataProducts contains a bunch of dicts...')
print(', '.join(dataProduct_Info), '\n')


print('What is in each of those dicts? They all have the same structure.')
dataProducts_ListOfDicts = TEAK_json['data']['dataProducts']
NEON_dataProducts_info = [str(subDict.keys())
                              for subDict in dataProducts_ListOfDicts] 
print('\n'.join(NEON_dataProducts_info), '\n')


print('What are the data product codes and titles?')
NEON_dataProductCodes_info = [str(productDict['dataProductCode'])+'\t'+str(productDict['dataProductTitle'])
                                  for productDict in dataProducts_ListOfDicts] 
print('\n'.join(NEON_dataProductCodes_info), '\n')

dict_keys(['data']) 

siteCode:	<class 'str'>
siteName:	<class 'str'>
siteDescription:	<class 'str'>
siteType:	<class 'str'>
siteLatitude:	<class 'float'>
siteLongitude:	<class 'float'>
stateCode:	<class 'str'>
stateName:	<class 'str'>
domainCode:	<class 'str'>
domainName:	<class 'str'>
deimsId:	<class 'str'>
releases:	<class 'list'>
dataProducts:	<class 'list'> 

Exploring dataProducts
dataProducts contains a bunch of dicts...
<class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class 'dict'>, <class '

# Filtering, downloading data

Using this information about the data available for my 3 sites of interest, I want to download data that meet the following specifications for each site:
* Data products:
    * Vegetation structure (DP1.10098.001)
    * Elevation - LiDAR (DP3.30024.001)
    * Spectrometer orthorectified surface directional reflectance - mosaic (DP3.30006.001)
* Most recent dataset

In [85]:
# Specifications
SERVER = 'http://data.neonscience.org/api/v0/'
SITES_LIST = ['TEAK', 'SOAP', 'SJER']
DATAPRODUCTS_DICT = {'Vegetation structure': 'DP1.10098.001', 
                     'Elevation - LiDAR': 'DP3.30024.001', 
                     'Spectrometer orthorectified surface directional reflectance - mosaic': 'DP3.30006.001'}

## Get a list of URLs for data to download ##

# get json info for my sites of interest
site_json_list = [requests.get(SERVER+'sites/'+site).json() 
                          for site in SITES_LIST]

# for each site, extract the relevant dataproducts
dataprod_codes_list = list(DATAPRODUCTS_DICT.values())

def getProductsAtSite(site_json, dataprod_codes_list):
    return [site_dataProd_dict
            for site_dataProd_dict in site_json['data']['dataProducts']
            if site_dataProd_dict['dataProductCode'] in dataprod_codes_list
           ]

site_dataprod_dict = {site_json['data']['siteCode']:
                      getProductsAtSite(site_json, dataprod_codes_list)
                      for site_json in site_json_list}

# get the URL and some metadata for only the most recent available datasets
## TODO

In [86]:
site_dataprod_dict

{'TEAK': [{'dataProductCode': 'DP1.10098.001',
   'dataProductTitle': 'Vegetation structure',
   'availableMonths': ['2015-08',
    '2015-09',
    '2015-10',
    '2021-09',
    '2021-11',
    '2021-12'],
   'availableDataUrls': ['https://data.neonscience.org/api/v0/data/DP1.10098.001/TEAK/2015-08',
    'https://data.neonscience.org/api/v0/data/DP1.10098.001/TEAK/2015-09',
    'https://data.neonscience.org/api/v0/data/DP1.10098.001/TEAK/2015-10',
    'https://data.neonscience.org/api/v0/data/DP1.10098.001/TEAK/2021-09',
    'https://data.neonscience.org/api/v0/data/DP1.10098.001/TEAK/2021-11',
    'https://data.neonscience.org/api/v0/data/DP1.10098.001/TEAK/2021-12'],
   'availableReleases': [{'release': 'PROVISIONAL',
     'availableMonths': ['2021-09', '2021-11', '2021-12']},
    {'release': 'RELEASE-2022',
     'availableMonths': ['2015-08', '2015-09', '2015-10']}]},
  {'dataProductCode': 'DP3.30006.001',
   'dataProductTitle': 'Spectrometer orthorectified surface directional reflect